In [36]:
import utils

import re
from collections import deque, defaultdict
from dataclasses import dataclass, field
from typing import Optional
from functools import lru_cache
import cProfile

## Day 11: Plutonian Pebbles

[#](https://adventofcode.com/2024/day/11) We have a line of stones, which change on each blink by:

* `0` -> `1`
* Even digits -> splits into two stones, don't keep extra leading 0's, e.g `1000 -> 10 and 0`
* if above doesn't apply, multiple stones number by 2024 

Linked lists! I thought I would need them for day 9, but got away with using a python list. This time around, as each blink will have a fair amount of insertions, I am going to attempt a proper linked list.


In [3]:
sample_input: str = """125 17"""

puzzle_input = utils.get_input(11, splitlines=False)

In [4]:
def parse_input(input_str=sample_input, debug: bool = False):
    numbers = [int(x) for x in re.findall(r"\d+", input_str)]
    if debug:
        print(f"{input_str} -> {numbers}")
    return numbers


data = parse_input(sample_input, True)

125 17 -> [125, 17]


In [5]:
@dataclass
class Stone:
    num: int
    prev: Optional["Stone"] = None
    next: Optional["Stone"] = None


s = Stone(3, prev=Stone(1), next=Stone(5))
s

Stone(num=3, prev=Stone(num=1, prev=None, next=None), next=Stone(num=5, prev=None, next=None))

In [6]:
def make_linked_list(data=data, debug=False):

    first = Stone(data[0])
    current = first

    for i, num in enumerate(data[1:]):
        new_stone = Stone(num, prev=current)
        current.next = new_stone
        current = new_stone

    return first


def print_stone(stone):
    nums = []
    while stone:
        nums.append(stone.num)
        stone = stone.next
    print(", ".join([str(n) for n in nums]))


first_stone = make_linked_list()
print(first_stone)
print_stone(first_stone)

Stone(num=125, prev=None, next=Stone(num=17, prev=..., next=None))
125, 17


In [7]:
def count_stones(first_stone):
    count = 1
    stone = first_stone
    while stone.next:
        count += 1
        stone = stone.next
    return count


count_stones(first_stone)

2

In [8]:
def evolve_stone(stone, multiple=2024, debug=False):
    """evolves stones, returns True if a stone was split"""
    if stone.num == 0:
        stone.num += 1
    elif (num_len := len(num_str := str(stone.num))) > 1 and num_len % 2 == 0:
        num_left = int(num_str[: num_len // 2])
        num_right = int(num_str[num_len // 2 :])
        if debug:
            print(f"Splitting {stone=} into {num_left=} {num_right=}")

        stone.num = num_left
        # insert new stone and fix prev and next links
        new_stone = Stone(num_right, prev=stone, next=stone.next)
        if stone.next:
            stone.next.prev = new_stone
        stone.next = new_stone
        return True
    else:
        stone.num *= multiple

    return False


stone = first_stone
while stone.next:
    # print(stone.num, evolve_stone(stone), stone.num)
    stone = stone.next

In [10]:
def solve(inp: str = sample_input, blinks=25, debug: bool = False):
    data = parse_input(inp)
    first_stone = make_linked_list(data)

    stone = first_stone
    if debug:
        print_stone(stone)

    for i in range(blinks):
        while stone:
            split = evolve_stone(stone, debug=False)
            if split:  # don't process the new stone in this blink
                stone = stone.next.next
            else:
                stone = stone.next
        stone = first_stone
        if debug:
            print_stone(stone)

    ans = count_stones(first_stone)

    return {"result": ans}


# checking sample ans
assert solve("0 1 10 99 999", blinks=1)["result"] == 7

solve(sample_input, blinks=6, debug=True)
assert solve(sample_input, blinks=6)["result"] == 22  # sample ans check
assert solve(sample_input, blinks=25)["result"] == 55312  # sample ans check

125, 17
253000, 1, 7
253, 0, 2024, 14168
512072, 1, 20, 24, 28676032
512, 72, 2024, 2, 0, 2, 4, 2867, 6032
1036288, 7, 2, 20, 24, 4048, 1, 4048, 8096, 28, 67, 60, 32
2097446912, 14168, 4048, 2, 0, 2, 4, 40, 48, 2024, 40, 48, 80, 96, 2, 8, 6, 7, 6, 0, 3, 2


In [11]:
%%time
results = solve(puzzle_input, debug=False)
print(f"Part 1: {results["result"]}") 

Part 1: 194482
CPU times: user 253 ms, sys: 4.45 ms, total: 258 ms
Wall time: 257 ms


## Part 2 - 75 blinks

Its the same, but exponentially bigger, so need to optmize. My solution above uses both a lot of memory and computation. We can improve both... thinking first to solve the memory part. Since each stone is fully independent of each other, lets try doing this one stone at a time.

Key changes:
1. `evolve_stone` uses ints
2. Using a dict to store each stone and the number of times it occurs. This is the key speedup - so we only need to evolve each kind of stone once and increment counts. This saves both space and computation.

In [58]:
@lru_cache(maxsize=None)
def evolve_stone(stone: int, multiple=2024):
    """takes a stone as int, evolves it and returns [stone] or [stone, new_stone]
    returns a list as sometimes 2 stones are returned"""
    if stone == 0:
        return [1]
    elif (num_len := len(num_str := str(stone))) > 1 and num_len % 2 == 0:
        num_left = int(num_str[: num_len // 2])
        num_right = int(num_str[num_len // 2 :])
        return [num_left, num_right]
    else:
        return [stone * multiple]


evolve_stone(125), evolve_stone(253000), evolve_stone(253), evolve_stone(512072)

([253000], [253, 0], [512072], [512, 72])

In [60]:
def solve_2(inp=sample_input, blinks=6, debug=False):
    data = parse_input(
        inp,
    )
    # store initial stone count in dict
    stone_counts = defaultdict(int)
    for stone in data:
        stone_counts[stone] += 1

    for blink in range(1, blinks + 1):
        new_counts = defaultdict(int)
        for stone, count in stone_counts.items():
            for blink_stone in evolve_stone(stone):
                new_counts[blink_stone] += count
        stone_counts = new_counts
        if debug:
            print(f"{blink}: {stone_counts} {sum(stone_counts.values())}")

    return sum(stone_counts.values())


assert solve_2(sample_input, 6, True) == 22

1: defaultdict(<class 'int'>, {253000: 1, 1: 1, 7: 1}) 3
2: defaultdict(<class 'int'>, {253: 1, 0: 1, 2024: 1, 14168: 1}) 4
3: defaultdict(<class 'int'>, {512072: 1, 1: 1, 20: 1, 24: 1, 28676032: 1}) 5
4: defaultdict(<class 'int'>, {512: 1, 72: 1, 2024: 1, 2: 2, 0: 1, 4: 1, 2867: 1, 6032: 1}) 9
5: defaultdict(<class 'int'>, {1036288: 1, 7: 1, 2: 1, 20: 1, 24: 1, 4048: 2, 1: 1, 8096: 1, 28: 1, 67: 1, 60: 1, 32: 1}) 13
6: defaultdict(<class 'int'>, {2097446912: 1, 14168: 1, 4048: 1, 2: 4, 0: 2, 4: 1, 40: 2, 48: 2, 2024: 1, 80: 1, 96: 1, 8: 1, 6: 2, 7: 1, 3: 1}) 22


In [61]:
%%time
solve_2(puzzle_input, 75)

CPU times: user 38.6 ms, sys: 1.25 ms, total: 39.8 ms
Wall time: 39.3 ms


232454623677743

Using dicts sped this up so fast, that lru_cache on `evolve_stone` isn't really necessary.